In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

#from rbm import RBM

## Define model

In [2]:
class RBM_simple():

    def __init__(self, num_visible, num_hidden, learning_rate=1e-3, use_cuda=False):
        """
        :param num_visible: int
            Number of visible nodes
        :param num_hidden: int
            Number of hidden nodes
        :param k:
            step of Gibbs sampling (Contrastive Divergence as optimizer). Usually = 1
        :param learning_rate: float
        :param use_cuda : Boolean
            Using GPU or not
        """
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        # self.k = k
        self.learning_rate = learning_rate
        self.use_cuda = use_cuda
        self.train_loss = 0

        self.weights = torch.randn(num_hidden, num_visible)
        self.visible_bias = torch.ones(num_visible)
        self.hidden_bias = torch.ones(num_hidden)

        if self.use_cuda:
            self.weights = self.weights.cuda()
            self.visible_bias = self.visible_bias.cuda()
            self.hidden_bias = self.hidden_bias.cuda()

    def sample_hidden(self, visible_prob):
        """
        :param visible_prob:
            Probability of visible layer
        :return:
            Hidden probability & Sample from hidden prob (Bernoulli distribution)
        """
        hidden_activations = torch.mm(visible_prob, self.weights.t()) + self.hidden_bias
        hidden_prob = torch.sigmoid(hidden_activations)
        return hidden_prob, torch.bernoulli(hidden_prob)

    def sample_visible(self, hidden_prob):
        """
        :param hidden_prob:
            Probability of hidden layer
        :return:
            Visible probability & Samples from visible prob (Bernoulli distribution)
        """
        visible_activations = torch.mm(hidden_prob, self.weights) + self.visible_bias
        visible_prob = torch.sigmoid(visible_activations)
        return visible_prob, torch.bernoulli(visible_prob)

    def train(self, v0, vk, ph0, phk):
        """
        :param v0:
        :param vk:
        :param ph0:
        :param phk:
        :return:
        """
        self.weights += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.visible_bias += torch.sum((v0 - vk), 0)
        self.hidden_bias += torch.sum((ph0 - phk), 0)
        self.train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        return self.train_loss

## Prepare datasets for train and test

In [3]:
movies = pd.read_csv('data/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('data/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('data/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [4]:
training_set = pd.read_csv('data/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('data/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int') 

In [5]:
no_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
no_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [6]:
def convert(data):
    new_data = []
    for id_users in range(1, no_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(no_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [7]:
training_set = convert(training_set)
test_set = convert(test_set)

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [9]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Start runner

In [10]:
nv = len(training_set[0])
nh = 200
batch_size = 100
rbm = RBM_simple(nv, nh)

In [11]:
nb_epoch = 5
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, no_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_hidden(v0)
        for k in range(10):
            _,hk = rbm.sample_hidden(vk)
            _,vk = rbm.sample_visible(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_hidden(vk)
        train_loss += rbm.train(v0, vk, ph0, phk)
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.9963)
epoch: 2 loss: tensor(4.4732)
epoch: 3 loss: tensor(6.6709)
epoch: 4 loss: tensor(8.9189)
epoch: 5 loss: tensor(11.1356)


In [12]:
test_loss = 0
s = 0.
for id_user in range(no_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_hidden(v)
        _,v = rbm.sample_visible(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2582)
